In [1]:
import os
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import keras
from keras import backend as K
import tensorflow as tf
from sklearn.model_selection import train_test_split
drive.mount('/content/drive/')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
#这里按自己的directory设置好了。我是mount在collab上以后做的。
#400张被preprocess好的图。np读进来就好。
data_loading_time_start = time.time()
dir_train_img = '/content/drive/My Drive/2007_training_images.npy'
dir_seg_img = '/content/drive/My Drive/2007_segmented_images.npy'
train_images = np.load(dir_train_img)
seg_images_21 = np.load(dir_seg_img)
print(f'Total time to load data: {time.time() - data_loading_time_start}s')
print(train_images.shape)
print(seg_images_21.shape)

Total time to load data: 11.511262655258179s
(400, 473, 473, 3)
(400, 473, 473, 21)


In [3]:
n_class = 21
input_img = keras.layers.Input(shape=(473,473,3))

conv1 = keras.layers.Conv2D(filters=64, kernel_size=(5,5), padding='same', data_format='channels_last', strides=3)(input_img)
bn1 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv1)
act1 = keras.layers.Activation(activation='relu')(bn1)
pool1 = keras.layers.MaxPool2D((2,2))(act1)

conv2 = keras.layers.Conv2D(filters=128, kernel_size=(4,4), padding='same', data_format='channels_last', strides=2)(pool1)
bn2 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv2)
act2 = keras.layers.Activation(activation='relu')(bn2)
pool2 = keras.layers.MaxPool2D((2,2))(act2)

conv3 = keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', data_format='channels_last')(pool2)
bn3 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv3)
act3 = keras.layers.Activation(activation='relu')(bn3)
pool3 = keras.layers.AveragePooling2D((2,2))(act3)

conv4 = keras.layers.Conv2D(filters=512, kernel_size=(2,2), padding='same', data_format='channels_last')(pool3)
bn4 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv4)
act4 = keras.layers.Activation(activation='relu')(bn4)
pool4 = keras.layers.AveragePooling2D((2,2))(act4)

channel_fusion = keras.layers.Conv2D(filters=200, kernel_size=(1,1), padding='same', data_format='channels_last')(pool4)

up1 = keras.layers.UpSampling2D((2,2))(channel_fusion)
conv5 = keras.layers.Conv2D(filters=512, kernel_size=(4,4), padding='same', data_format='channels_last')(up1)
bn5 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv5)
act5 = keras.layers.Activation(activation='relu')(bn5)

up2 = keras.layers.UpSampling2D((3,3))(act5)
conv6 = keras.layers.Conv2D(filters=256, kernel_size=(2,2), padding='same', data_format='channels_last')(up2)
bn6 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv6)
act6 = keras.layers.Activation(activation='relu')(bn6)

up3 = keras.layers.UpSampling2D((4,4))(act6)
conv7 = keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', data_format='channels_last')(up3)
bn7 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv7)
act7 = keras.layers.Activation(activation='relu')(bn7)

up4 = keras.layers.UpSampling2D((2,2))(act7)
conv8 = keras.layers.Conv2D(filters=64, kernel_size=(4,4), padding='valid', data_format='channels_last')(up4)
bn8 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv8)
act8 = keras.layers.Activation(activation='relu')(bn8)

up5 = keras.layers.UpSampling2D((2,2))(act8)
conv9 = keras.layers.Conv2D(filters=n_class, kernel_size=(2,2), padding='valid', data_format='channels_last')(up5)
bn9 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv9)
output_img = keras.layers.Activation(activation='softmax')(bn9)

In [4]:
FCNN = keras.models.Model(input_img, output_img)
print(FCNN.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 473, 473, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 158, 158, 64)      4864      
_________________________________________________________________
batch_normalization_1 (Batch (None, 158, 158, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 158, 158, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 79, 79, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 40, 40, 128)       131200    
_________________________________________________________________
batch_normalization_2 (Batch (None, 40, 40, 128)       512 

In [0]:
opt = keras.optimizers.adam(lr = 0.08)

In [6]:
FCNN.compile(
    loss='categorical_crossentropy',
    optimizer =opt,
    metrics=['categorical_accuracy'])

In [0]:
X = np.array(train_images)
Y = np.array(seg_images_21)

In [8]:
FCNN.fit(X, Y, epochs = 50, batch_size = 8, validation_split=.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 320 samples, validate on 80 samples
Epoch 1/50
320/320 [==============================] - 18s 58ms/step - loss: 0.7417 - categorical_accuracy: 0.0600 - val_loss: 1.2056 - val_categorical_accuracy: 0.0383
Epoch 2/50
320/320 [==============================] - 12s 38ms/step - loss: 0.6841 - categorical_accuracy: 0.0696 - val_loss: 0.6500 - val_categorical_accuracy: 0.0490
Epoch 3/50
320/320 [==============================] - 12s 38ms/step - loss: 0.6765 - categorical_accuracy: 0.0695 - val_loss: 0.6958 - val_categorical_accuracy: 0.0396
Epoch 4/50
320/320 [==============================] - 12s 38ms/step - loss: 0.6646 - categorical_accuracy: 0.0726 - val_loss: 0.7326 - val_categorical_accuracy: 0.0373
Epoch 5/50
320/320 [==============================] - 12s 38ms/step - loss: 0.6618 - categorical_accuracy: 0.0717 - val_loss: 1.2533 - val_categorical_accuracy: 0.0100
Epoch 6/50
320/320 

In [0]:
FCNN.save('/content/drive/My Drive/FCNN.h5')

In [0]:
# y_hat = FCNN.predict(x_test)

In [0]:
# print(y_hat[:5])